## 🐍 Python 텍스트 분석: 최신 Transformer 임베딩 모델 활용하기 (Part 4)

이전 파트에서는 텍스트를 숫자로 표현하는 고전적인 벡터화 기법(BoW, TF-IDF, Word2Vec, FastText)들을 학습했습니다. 이번 시간에는 여기서 한 걸음 더 나아가, 현대 자연어 처리(NLP)의 핵심 기술인 **Transformer 기반 임베딩 모델**을 사용하여 텍스트의 '의미'를 벡터에 담아내는 방법을 배웁니다.

로컬 환경에 AI 모델을 손쉽게 배포할 수 있는 **Ollama**와 **Docker**를 활용하여, 최신 오픈소스 모델로 텍스트를 임베딩하고 그 결과를 실제 문제에 적용하는 실전적인 경험을 하게 될 것입니다.

---

### 1. 왜 Transformer 임베딩인가?: 문맥을 이해하는 AI

#### 💡 개념 (Concept)

Word2Vec이나 FastText도 훌륭한 단어 임베딩 기법이지만, 결정적인 한계를 가집니다. 바로 **문맥에 따라 단어의 의미가 달라지는 현상**을 아주 효과적으로 반영하지 못한다는 점입니다. 예를 들어, "사과를 먹었다"와 "진심으로 사과했다"에서 '사과'는 완전히 다른 의미이지만, Word2Vec에서는 동일한 벡터로 표현됩니다.

**Transformer 기반 임베딩 모델**은 `어텐션(Attention)` 메커니즘을 사용하여 이 문제를 해결합니다. 

문장 전체의 구조와 단어 간의 관계를 종합적으로 파악하여, 같은 단어라도 문맥에 따라 다른 의미를 가진 벡터를 생성합니다.

* **문맥적 임베딩 (Contextual Embedding)**: 단어가 문장 내에서 어떤 의미로 쓰였는지 파악하여 벡터를 생성합니다.
* **사전 학습된 거대 언어 모델 (Pre-trained LLMs)**: 방대한 양의 텍스트 데이터로 미리 학습된 모델을 활용하여, 높은 수준의 언어 이해 능력을 보여줍니다.
* **전이 학습 (Transfer Learning)**: 잘 학습된 모델을 우리의 특정 작업(유사도 계산, 검색, 분류 등)에 바로 적용하여 높은 성능을 낼 수 있습니다.

이번 학습자료에서는 현재 가장 뛰어난 오픈소스 임베딩 모델 중 하나인 `nomic-embed-text`를 Ollama를 통해 로컬 환경에서 직접 활용해봅니다.

---

### 2. Ollama와 Docker를 이용한 로컬 AI 환경 구축

#### 💡 개념 (Concept)

**Ollama**는 강력한 언어 모델들을 자신의 컴퓨터에서 간편하게 설치하고 실행할 수 있게 해주는 도구입니다. **Docker**는 애플리케이션을 신속하게 구축, 테스트 및 배포할 수 있는 컨테이너 기술입니다. 이 둘을 함께 사용하면, 복잡한 설치 과정 없이 단 몇 줄의 명령어로 나만의 AI 서버를 구축하고 모델을 실행할 수 있습니다.

* **`docker-compose.yml`**: 여러 개의 Docker 컨테이너 설정을 하나의 파일로 정의하고 관리하게 해주는 도구입니다. 우리는 이 파일을 사용해 Ollama 서버를 설정합니다.
* **볼륨(Volume) 마운트**: Docker 컨테이너가 삭제되어도 데이터를 보존하기 위해, 로컬 디렉터리와 컨테이너 내부 디렉터리를 연결합니다. 이렇게 하면 다운로드한 모델이 영구적으로 저장됩니다.

#### 💻 예시 코드 (Example Code)

먼저, 프로젝트를 진행할 폴더를 생성하고 `docker-compose.yml` 파일을 작성합니다.

```bash
# 1. 프로젝트 폴더 생성 및 이동
mkdir my-embedding-server
cd my-embedding-server

# 2. docker-compose.yml 파일 생성 (아래 내용 복사)
```

**`docker-compose.yml`**

```yaml
version: '3.8'

services:
  ollama:
    image: ollama/ollama:latest
    container_name: ollama_server
    ports:
      - "11434:11434"
    volumes:
      - ./ollama_data:/root/.ollama
    restart: unless-stopped
```

이제 터미널에서 아래 명령어로 Ollama 서버를 시작합니다.

```bash
# Docker 컨테이너를 백그라운드에서 실행
docker-compose up -d
```

#### ✏️ 연습 문제 (Practice Problems)

1.  터미널에서 `docker ps` 명령어를 실행하여 `ollama_server` 컨테이너가 정상적으로 실행 중인지 확인하세요.
2.  `docker-compose logs -f` 명령어를 실행하여 Ollama 서버의 실시간 로그를 확인해 보세요. 어떤 내용이 출력되나요? (종료는 `Ctrl + C`)

---

### 3. 고성능 임베딩 모델 설치 및 API 호출

#### 💡 개념 (Concept)

이제 실행 중인 Ollama 서버에 텍스트 임베딩을 위한 모델을 설치할 차례입니다. `nomic-embed-text` 모델은 다양한 언어를 지원하며 특히 한국어 성능이 우수하여 MTEB(Massive Text Embedding Benchmark)에서 높은 순위를 차지하고 있습니다.

Ollama 서버는 HTTP API를 제공하므로, Python의 `requests` 라이브러리를 사용해 간단하게 텍스트를 보내고 임베딩 벡터를 받아올 수 있습니다. Transformer 모델은 입력 텍스트를 내부적으로 자신만의 토큰화 방식으로 처리하므로, `kiwipiepy` 같은 외부 토크나이저를 사용할 필요가 없습니다.

#### 💻 예시 코드 (Example Code)

먼저, Docker 컨테이너 안에서 `nomic-embed-text` 모델을 다운로드합니다.

```bash
# 'ollama_server' 컨테이너 안에서 'ollama pull' 명령 실행
docker-compose exec ollama_server ollama pull nomic-embed-text
```

모델 설치가 완료되면, Python 코드로 임베딩 벡터를 생성합니다.

In [1]:
import requests
import json
import numpy as np

OLLAMA_ENDPOINT = "http://localhost:11434/api/embeddings"
DEFAULT_MODEL = "nomic-embed-text"

def get_embedding(text: str, model: str = DEFAULT_MODEL) -> np.ndarray | None:
    """
    주어진 텍스트를 Ollama API를 사용하여 임베딩 벡터로 변환합니다.

    Args:
        text (str): 임베딩할 텍스트
        model (str): 사용할 모델 이름

    Returns:
        np.ndarray | None: 임베딩 벡터 (NumPy 배열) 또는 에러 발생 시 None
    """
    try:
        payload = {"model": model, "prompt": text}
        response = requests.post(OLLAMA_ENDPOINT, json=payload)
        response.raise_for_status()  # HTTP 2xx 이외의 응답 코드는 예외 발생

        embedding_data = response.json()["embedding"]
        return np.array(embedding_data)

    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 에러가 발생했습니다: {e}")
        return None
    except KeyError:
        print("응답 데이터에서 'embedding' 키를 찾을 수 없습니다.")
        return None

sample_text = "인공지능은 세상을 어떻게 바꾸고 있을까?"
embedding_vector = get_embedding(sample_text)

if embedding_vector is not None:
    print(f"입력 텍스트: \"{sample_text}\"")
    print(f"사용한 모델: {DEFAULT_MODEL}")
    print(f"벡터 차원 수: {len(embedding_vector)}")
    print(f"임베딩 벡터 (앞 5개 차원): {embedding_vector[:5]}")

입력 텍스트: "인공지능은 세상을 어떻게 바꾸고 있을까?"
사용한 모델: nomic-embed-text
벡터 차원 수: 768
임베딩 벡터 (앞 5개 차원): [-0.40841702  0.15448135 -2.94594789 -0.00443408 -0.10458014]


#### ✏️ 연습 문제 (Practice Problems)

1.  자신이 좋아하는 영화나 책의 한 구절을 `sample_text` 변수에 넣어 임베딩 벡터를 생성하고, 벡터의 차원 수를 확인해 보세요.
2.  `mxbai-embed-large` 모델도 한국어 성능이 좋은 모델 중 하나입니다. `docker-compose exec ollama_server ollama pull mxbai-embed-large` 명령어로 모델을 설치한 뒤, `get_embedding` 함수를 호출할 때 `model` 인자를 `"mxbai-embed-large"`로 지정하여 임베딩을 생성하고 벡터 차원 수가 어떻게 다른지 비교해 보세요.

---

### 4. 임베딩 벡터 활용: 의미 기반 유사도 계산 및 시각화

#### 💡 개념 (Concept)

임베딩의 가장 강력한 활용 분야는 **의미 기반 검색(Semantic Search)** 입니다. 텍스트의 의미가 벡터 공간상의 좌표로 표현되므로, 두 벡터 사이의 거리가 가까울수록 의미가 유사하다고 판단할 수 있습니다.

**코사인 유사도(Cosine Similarity)** 는 두 벡터가 이루는 각도의 코사인 값을 이용하여 유사도를 측정하는 방법입니다. 벡터의 크기와 관계없이 방향의 유사성만 보기 때문에 텍스트 유사도 계산에 널리 쓰입니다. 값의 범위는 -1에서 1 사이이며, 1에 가까울수록 유사도가 높습니다.

$$ \text{Cosine Similarity}(\vec{A}, \vec{B}) = \frac{\vec{A} \cdot \vec{B}}{\|\vec{A}\| \|\vec{B}\|} $$

#### 💻 예시 코드 (Example Code)

다양한 주제의 문장들을 임베딩하고, 그들 간의 코사인 유사도를 계산하여 시각화해 보겠습니다.

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from numpy.linalg import norm

def cosine_similarity(vec_a: np.ndarray, vec_b: np.ndarray) -> float:
    """두 NumPy 벡터 간의 코사인 유사도를 계산합니다."""
    return np.dot(vec_a, vec_b) / (norm(vec_a) * norm(vec_b))

# 분석할 문장들
sentences = [
    "오늘 점심 뭐 먹지?",
    "배가 고픈데 맛있는 메뉴 추천해줘.",
    "요즘 볼만한 영화가 있을까?",
    "최근에 개봉한 액션 영화 재미있더라.",
    "파이썬 코딩 너무 재미있어.",
    "이 버그는 어떻게 해결해야 할까?",
]

# 모든 문장을 임베딩 벡터로 변환
embedding_vectors = [get_embedding(s) for s in sentences]
embedding_vectors[:10]

[array([-6.04353130e-01, -1.29349279e+00, -2.86517406e+00, -1.58960417e-01,
         7.33030513e-02, -3.24223071e-01, -6.63400114e-01, -2.49603838e-01,
        -3.61440629e-01, -9.72959474e-02,  4.68710408e-04,  1.84840202e+00,
         1.61439526e+00, -3.78451467e-01, -3.66064042e-01, -7.43826330e-01,
        -1.23047209e+00, -1.34781373e+00, -7.48044848e-01,  1.13161933e+00,
         2.45047808e-01,  5.77197075e-01, -1.30934453e+00, -1.20049071e+00,
         2.90006042e+00,  2.51267791e-01,  7.35598207e-01,  1.54659361e-01,
        -7.31934309e-01, -3.61061424e-01,  6.03400648e-01, -1.75303435e+00,
         3.57788593e-01, -8.05130422e-01, -1.37105572e+00, -7.64342904e-01,
         7.04149529e-02,  3.63521993e-01,  9.57091749e-01,  4.42821980e-01,
         1.62567254e-02, -1.10250390e+00, -6.99169338e-01, -5.53681850e-01,
         8.17437172e-01,  1.19019568e+00, -2.80409098e-01, -3.51339072e-01,
         1.54131067e+00, -9.01834428e-01,  9.34935883e-02,  1.30492849e-02,
         1.2

In [3]:
# 유사도 행렬 생성
num_sentences = len(sentences)
similarity_matrix = np.zeros((num_sentences, num_sentences))

for i in range(num_sentences):
    for j in range(num_sentences):
        if embedding_vectors[i] is not None and embedding_vectors[j] is not None:
            similarity_matrix[i, j] = cosine_similarity(embedding_vectors[i], embedding_vectors[j])

# DataFrame으로 변환
similarity_df = pd.DataFrame(similarity_matrix, index=sentences, columns=sentences)

# Plotly를 사용한 히트맵 시각화
fig = px.imshow(similarity_df,
                text_auto=".2f",
                title="문장 간 코사인 유사도 (Cosine Similarity)",
                color_continuous_scale="Viridis")
fig.update_xaxes(side="top")
fig.show()

#### ✏️ 연습 문제 (Practice Problems)

1.  위 `sentences` 리스트에 스포츠(예: "손흥민 선수가 골을 넣었다."), 날씨(예: "내일은 비가 올 예정입니다.") 등 완전히 다른 주제의 문장 2개를 추가하여 유사도 행렬을 다시 계산하고 시각화해 보세요. 결과 히트맵에서 어떤 변화가 관찰되나요?
2.  `get_embedding` 함수와 `cosine_similarity` 함수를 활용하여, 하나의 **질의(query)** 문장과 여러 개의 **문서(document)** 문장들 사이의 유사도를 각각 계산하고, 가장 유사도가 높은 문서를 찾아 출력하는 `find_most_similar` 함수를 작성해 보세요.


---

### 5. 🚀 실전 프로젝트: 영화 리뷰 의미 기반 검색기 만들기

이제까지 배운 내용을 총망라하여, 사용자의 질의에 가장 잘 맞는 영화 리뷰를 찾아주는 간단한 의미 기반 검색기를 만들어 보겠습니다. 이 프로젝트는 키워드 매칭의 한계를 넘어, 문맥적 의미를 파악하여 검색 결과를 제공하는 경험을 선사할 것입니다.

#### 💡 프로젝트 개요 (Project Overview)

1.  **데이터 준비**: 영화 리뷰 데이터셋(코퍼스)을 준비합니다.
2.  **문서 임베딩**: 모든 영화 리뷰 문서를 미리 임베딩하여 벡터 데이터베이스(여기서는 간단히 리스트)를 구축합니다. 실제 서비스에서는 이 벡터들을 DB에 저장해두고 사용합니다.
3.  **검색 함수 구현**: 사용자로부터 검색 질의를 입력받아 임베딩합니다.
4.  **유사도 계산 및 랭킹**: 질의 벡터와 모든 문서 벡터 간의 코사인 유사도를 계산하여 가장 점수가 높은 순서대로 리뷰를 정렬하여 보여줍니다.

#### 💻 예시 코드 (Example Code)

In [ ]:
import numpy as np
import pandas as pd

# 1. 데이터 준비 (이전 파트의 영화 리뷰 데이터 활용)
corpus = [
    '배우의 연기력이 정말 대단한 영화였어요.',
    '스토리가 너무 예측 가능해서 연기력이 아까웠다.',
    '감독의 연출과 배우의 연기가 조화로웠던 영화.',
    '와 이 영화 진짜 대박이야! 배우들 연기 미쳤고 스토리도 완전 몰입됨',
    '음... 좀 아쉽네요. 감독이 뭘 말하고 싶었는지 모르겠어요',
    '연기는 괜찮았는데 결말이 너무 뻔해서 실망했습니다',
    '헐 이거 완전 꿀잼ㅋㅋ 예상 못한 반전에 소름돋았어',
    '감독님... 제발 좀 더 신경써서 찍으시길... 연출이 엉망이에요',
    '주연배우 연기 진짜 자연스럽더라! 몰입도 최고였음',
    '스토리가 조금 복잡하긴 했지만 나름 볼만했어요',
    '이런 영화를 왜 만들었는지 이해가 안 가네... 시간 아까움',
    '배우들 케미 완전 좋았고 연출도 깔끔했음. 추천!',
    '예측할 수 없는 전개로 끝까지 긴장감 넘쳤습니다',
    '연기력은 인정하지만 스토리가 너무 뻔해서... 그냥 그래요',
    '감독의 의도는 좋았으나 표현 방식이 아쉬웠네요',
    'ㅋㅋㅋ 이거 뭐야 완전 재밌잖아? 배우들 연기 ㄹㅇ 대단함',
    '조용한 영화인데 배우들 연기가 워낙 좋아서 지루하지 않았어요',
    '액션은 별로였지만 인간관계 드라마가 탄탄해서 만족',
    '아 진짜... 왜 이렇게 만들었을까? 감독 뭐하는 거야',
    '처음엔 지루했는데 중반부터 완전 몰입! 연출 센스 있네',
    '배우들 연기는 좋았지만 전체적으로 밋밋한 느낌이에요',
    '와... 이런 스토리는 처음 봐! 정말 신선하고 감동적이었어',
    '연출과 연기 모두 완벽했습니다. 올해 최고의 작품 중 하나!',
    '뭔가 아쉬운 부분들이 있지만 그래도 볼만한 영화였어요'
]

# 2. 모든 문서 임베딩 (시간이 걸릴 수 있습니다)
print("영화 리뷰 문서를 임베딩 중입니다...")
doc_embeddings = [get_embedding(doc) for doc in corpus]
# 실패한 임베딩(None)이 있는 경우를 대비하여 필터링
valid_embeddings_data = [
    (emb, doc) for emb, doc in zip(doc_embeddings, corpus) if emb is not None
]
doc_embeddings, corpus = zip(*valid_embeddings_data)
print("임베딩 완료!")


# 3. 의미 기반 검색 함수 구현
def semantic_search(query: str, top_n: int = 5):
    """
    질의와 가장 유사한 문서를 찾아 상위 n개를 반환합니다.
    """
    query_embedding = get_embedding(query)
    if query_embedding is None:
        print("질의를 임베딩하는 데 실패했습니다.")
        return

    # 유사도 계산
    similarities = [cosine_similarity(query_embedding, doc_emb) for doc_emb in doc_embeddings]

    # 유사도와 함께 문서 저장 후 정렬
    results = sorted(zip(similarities, corpus), key=lambda x: x[0], reverse=True)

    # 상위 n개 결과 출력
    print(f"\\n--- 검색 결과 (질의: '{query}') ---")
    for i, (score, doc) in enumerate(results[:top_n], 1):
        print(f"{i}위 (유사도: {score:.4f}): {doc}")

# 4. 검색기 실행
if __name__ == "__main__":
    search_query1 = "가슴 따뜻해지는 배우들의 열연"
    semantic_search(search_query1)

    search_query2 = "시간 가는 줄 모르고 봤어요"
    semantic_search(search_query2)

    search_query3 = "지루하고 실망스러운 플롯"
    semantic_search(search_query3)

#### ✏️ 연습 문제 (Practice Problems)

1.  위 `semantic_search` 함수를 사용하여 "반전이 있는 스릴러" 또는 "가족과 함께 볼만한 영화"와 같이 더 구체적인 내용으로 검색을 수행하고, 그 결과를 분석해 보세요.


2.  현재 검색 결과는 유사도 점수와 문서 내용만 보여줍니다. `semantic_search` 함수의 출력 부분을 수정하여, 각 결과 문서의 인덱스 번호(corpus에서의 위치)도 함께 출력되도록 만들어 보세요.


---

### 💡 사용 Tip (Usage Tips)

* Ollama 서버는 한 번 실행해두면 계속해서 사용할 수 있습니다. 컴퓨터를 재부팅하면 Docker가 자동으로 컨테이너를 재시작합니다.
* 실제 대규모 서비스에서는 매번 모든 문서의 임베딩을 계산하는 것은 비효율적입니다. 미리 모든 문서의 벡터를 계산하여 FAISS나 ChromaDB 같은 벡터 데이터베이스에 저장해두고, 질의 벡터와 DB에 저장된 벡터들 간의 유사도를 빠르게 찾는 방식을 사용합니다.